In [ ]:
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

In [ ]:
!pip install git+https://github.com/ahendriksen/tomosipo.git

In [ ]:
import numpy as np
print(np.__version__)

In [ ]:
import os
import astra
import tomosipo as ts
from matplotlib import pyplot as plt
from functions.dataset import LoDoPaBDataset

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Testing good data access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_path = "/content/drive/My Drive/"
files = os.listdir(drive_path)
files

In [ ]:
# Testing it reads from the correct path
test_path = os.path.join(drive_path,"ground_truth_test")
test_set = LoDoPaBDataset(test_path)
print("length of the SimpleDataset object: ", len(test_set))

 ## Testing indexis and sinogram creation

In [ ]:
# Get the first sample from the dataset
sample = test_set[1]

# Extract the ground truth image and the sinogram
ground_truth_image = sample['ground_truth'][0].squeeze().numpy()  # Remove batch dim and convert to NumPy
sinogram_image = sample['sinogram'][0].squeeze().numpy()
noisy_sinogram_image = sample['noisy_sinogram'][0].squeeze().numpy()
single_back_projection = sample['single_back_projections']

# Print the shapes
print("Ground Truth Image Shape:", ground_truth_image.shape)
print("Sinogram Image Shape:", sinogram_image.shape)
print("Noisy Sinogram Image Shape:", noisy_sinogram_image.shape)
print("Single Backprojected Images Shape:", single_back_projection.shape)

# Plot the images
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

# Ground truth image
axes[0].imshow(ground_truth_image, cmap='gray')
axes[0].set_title("Ground Truth Image")
axes[0].axis("off")

# Sinogram image
axes[1].imshow(sinogram_image, cmap='gray',aspect='auto')
axes[1].set_title("Sinogram Image")
axes[1].axis("off")

# Noisy Sinogram image
axes[2].imshow(noisy_sinogram_image, cmap='gray',aspect='auto')
axes[2].set_title("Noisy Sinogram Image")
axes[2].axis("off")

# Single-Backprojected image
axes[3].imshow(single_back_projection[4], cmap='gray') # 0 vertical, around 7-8 horizontal
axes[3].set_title("First Single-Backprojected Image")
axes[3].axis("off")

# Show the plot
plt.show()

NameError: name 'test_set' is not defined

In [ ]:
# read first file of test data
file_path= os.path.join(test_path, "ground_truth_test_000.hdf5")
with h5py.File(file_path, 'r') as f:
            ground_truth = f['data'][:]

# Convert to tensor
sample = torch.tensor(ground_truth, dtype=torch.float32)

# Taking the first image to test if the dataset took it correctly
sample_image = sample[1]/1000

#Visualize one image
plt.imshow(sample_image, cmap='gray')
plt.colorbar()
plt.show()